# Calcul des coordonnées des variables météorologiques

In [1]:
import pandas as pd
import geopandas as gpd
import os
import numpy as np
from shapely.geometry import Point

In [2]:
# Confirmation d'être à la racine du dossier
project_root = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
os.chdir(project_root)

In [19]:
# Import de la data
puiss_solaire = gpd.read_file("data/processed/occ_puiss_solaire_installee_iris.gpkg")
puiss_solaire["maxPuis"] = puiss_solaire["maxPuis"].astype(float)

In [20]:
def compute_barycentre(lat: np.ndarray, lon: np.ndarray, weights: np.ndarray) -> np.ndarray:
    """Calcule le barycentre pondéré par un poids pour une zone donnée

    Args:
        lat (np.ndarray): Toutes les latitudes des objets qui possèdent un poids
        long (np.ndarray):  Toutes les longitudes des objets qui possèdent un poids
        weights (np.ndarray): Poids assignés aux objets

    Returns:
        np.ndarray: latitude et longitude du barycentre
    """
    assert lat.shape == lon.shape == weights.shape

    return np.array([np.sum(lat*weights)/np.sum(weights), np.sum(lon*weights)/np.sum(weights)])

In [21]:
def search_weather_coordinates(gdf: gpd.GeoDataFrame, weights: np.ndarray):
    
    """_summary_

    Args:
        gdf (gpd.GeoDataFrame): _description_
        weights (np.ndarray): _description_

    Returns:
        _type_: _description_
    """

    # Creation des features
    centroid = gdf.centroid
    x = np.array(centroid.x)
    y = np.array(centroid.y)

    barycentre_coord = compute_barycentre(x, y, weights)

    return barycentre_coord

In [22]:
puiss_solaire.head()

,cleabs,code_insee,nom_commune,iris,code_iris,nom_iris,type_iris,codeDepartement,codeRegion,dateMiseEnService,filiere,debitMaximal,maxPuis,geometry
0,IRIS____0000000112030101,11203,Lézignan-Corbières,0101,112030101,Zone d'Activités,A,11,76,05/11/2009,Solaire,0.0,168.94,"MULTIPOLYGON (((681614.1 6228809.3, 681235.6 6..."
1,IRIS____0000000314330102,31433,Portet-sur-Garonne,0102,314330102,Zone d'Activités,A,31,76,27/07/2011,Solaire,0.0,135.00,"MULTIPOLYGON (((568930.3 6269968.2, 568893.9 6..."
2,IRIS____0000000661720102,66172,Saint-Estève,0102,661720102,Zone d'Activités,A,66,76,05/08/2011,Solaire,0.0,108.30,"MULTIPOLYGON (((684756.5 6180601, 684832.5 618..."
3,IRIS____0000000652580103,65258,Lannemezan,0103,652580103,Quartier Demi-Lune,A,65,76,01/12/2023,Solaire,0.0,250.00,"MULTIPOLYGON (((488692.5 6229018.2, 488639.6 6..."
4,IRIS____0000000092250103,09225,Pamiers,0103,092250103,Z.I. de Pic,A,09,76,12/04/2013,Solaire,0.0,100.00,"MULTIPOLYGON (((587623.3 6224907.7, 587610.9 6..."


In [25]:
# Calcul du barycentre représentant le scénario central
central_coord = Point(search_weather_coordinates(puiss_solaire, puiss_solaire["maxPuis"]))
print(central_coord)

POINT (646749.197872009 6280856.568014578)


In [26]:
dict_coord = {}
for departement in puiss_solaire["codeDepartement"].unique():
    temp_gdf = puiss_solaire[puiss_solaire["codeDepartement"]==departement]
    dep_coord = search_weather_coordinates(temp_gdf, temp_gdf["maxPuis"])
    dict_coord[departement] = Point(dep_coord)

In [ ]:
dict_coord[1000] = central_coord # Scénario central

In [ ]:
s_gdf = gpd.GeoDataFrame(pd.DataFrame.from_dict(dict_coord, orient='index', columns=["geometry"]), geometry="geometry", crs="EPSG:2154") # Création du GeoDataFrame pour illustration
s_gdf = s_gdf.to_crs(4326) # Coordonnées de Mercator pour l'API
print(s_gdf)

                      geometry
11    POINT (2.44042 43.19964)
31    POINT (1.28317 43.48915)
66    POINT (2.72224 42.62486)
65    POINT (0.18671 43.23553)
09     POINT (1.5799 43.07568)
34    POINT (3.54115 43.55647)
81     POINT (2.10443 43.7987)
82    POINT (1.24372 43.98832)
12      POINT (2.59696 44.278)
32     POINT (0.5125 43.71799)
30    POINT (4.37733 43.97766)
48     POINT (3.28697 44.4936)
46    POINT (1.65832 44.72196)
1000  POINT (2.34041 43.62476)


In [35]:
s_gdf.to_file("data/processed/weather_coordinates.gpkg")